# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa068139d60>
├── 'a' --> tensor([[-0.1981,  1.5641, -1.2432],
│                   [-0.9128,  1.3244,  0.2030]])
└── 'x' --> <FastTreeValue 0x7fa068139df0>
    └── 'c' --> tensor([[-0.0952, -0.4260, -0.0629, -0.9232],
                        [-1.0206,  0.2144,  1.0818, -1.4281],
                        [-1.5229,  1.0045, -0.6346, -0.7571]])

In [4]:
t.a

tensor([[-0.1981,  1.5641, -1.2432],
        [-0.9128,  1.3244,  0.2030]])

In [5]:
%timeit t.a

62.5 ns ± 0.0125 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa068139d60>
├── 'a' --> tensor([[ 0.4176,  0.5906, -0.8900],
│                   [ 2.1340, -0.8521, -0.0562]])
└── 'x' --> <FastTreeValue 0x7fa068139df0>
    └── 'c' --> tensor([[-0.0952, -0.4260, -0.0629, -0.9232],
                        [-1.0206,  0.2144,  1.0818, -1.4281],
                        [-1.5229,  1.0045, -0.6346, -0.7571]])

In [7]:
%timeit t.a = new_value

62 ns ± 0.0343 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1981,  1.5641, -1.2432],
               [-0.9128,  1.3244,  0.2030]]),
    x: Batch(
           c: tensor([[-0.0952, -0.4260, -0.0629, -0.9232],
                      [-1.0206,  0.2144,  1.0818, -1.4281],
                      [-1.5229,  1.0045, -0.6346, -0.7571]]),
       ),
)

In [10]:
b.a

tensor([[-0.1981,  1.5641, -1.2432],
        [-0.9128,  1.3244,  0.2030]])

In [11]:
%timeit b.a

55.5 ns ± 0.0624 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9916,  1.2574,  1.4500],
               [-0.0970, -0.5021, -0.5272]]),
    x: Batch(
           c: tensor([[-0.0952, -0.4260, -0.0629, -0.9232],
                      [-1.0206,  0.2144,  1.0818, -1.4281],
                      [-1.5229,  1.0045, -0.6346, -0.7571]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.218 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

874 ns ± 6.36 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 15.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 619 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 183 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9fbcdbc0a0>
├── 'a' --> tensor([[[-0.1981,  1.5641, -1.2432],
│                    [-0.9128,  1.3244,  0.2030]],
│           
│                   [[-0.1981,  1.5641, -1.2432],
│                    [-0.9128,  1.3244,  0.2030]],
│           
│                   [[-0.1981,  1.5641, -1.2432],
│                    [-0.9128,  1.3244,  0.2030]],
│           
│                   [[-0.1981,  1.5641, -1.2432],
│                    [-0.9128,  1.3244,  0.2030]],
│           
│                   [[-0.1981,  1.5641, -1.2432],
│                    [-0.9128,  1.3244,  0.2030]],
│           
│                   [[-0.1981,  1.5641, -1.2432],
│                    [-0.9128,  1.3244,  0.2030]],
│           
│                   [[-0.1981,  1.5641, -1.2432],
│                    [-0.9128,  1.3244,  0.2030]],
│           
│                   [[-0.1981,  1.5641, -1.2432],
│                    [-0.9128,  1.3244,  0.2030]]])
└── 'x' --> <FastTreeValue 0x7f9fbcd9d160>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 352 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa0680d8460>
├── 'a' --> tensor([[-0.1981,  1.5641, -1.2432],
│                   [-0.9128,  1.3244,  0.2030],
│                   [-0.1981,  1.5641, -1.2432],
│                   [-0.9128,  1.3244,  0.2030],
│                   [-0.1981,  1.5641, -1.2432],
│                   [-0.9128,  1.3244,  0.2030],
│                   [-0.1981,  1.5641, -1.2432],
│                   [-0.9128,  1.3244,  0.2030],
│                   [-0.1981,  1.5641, -1.2432],
│                   [-0.9128,  1.3244,  0.2030],
│                   [-0.1981,  1.5641, -1.2432],
│                   [-0.9128,  1.3244,  0.2030],
│                   [-0.1981,  1.5641, -1.2432],
│                   [-0.9128,  1.3244,  0.2030],
│                   [-0.1981,  1.5641, -1.2432],
│                   [-0.9128,  1.3244,  0.2030]])
└── 'x' --> <FastTreeValue 0x7fa0680d8910>
    └── 'c' --> tensor([[-0.0952, -0.4260, -0.0629, -0.9232],
                        [-1.0206,  0.2144,  1.0818, -1.4281],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.5 µs ± 54.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1981,  1.5641, -1.2432],
                [-0.9128,  1.3244,  0.2030]],
       
               [[-0.1981,  1.5641, -1.2432],
                [-0.9128,  1.3244,  0.2030]],
       
               [[-0.1981,  1.5641, -1.2432],
                [-0.9128,  1.3244,  0.2030]],
       
               [[-0.1981,  1.5641, -1.2432],
                [-0.9128,  1.3244,  0.2030]],
       
               [[-0.1981,  1.5641, -1.2432],
                [-0.9128,  1.3244,  0.2030]],
       
               [[-0.1981,  1.5641, -1.2432],
                [-0.9128,  1.3244,  0.2030]],
       
               [[-0.1981,  1.5641, -1.2432],
                [-0.9128,  1.3244,  0.2030]],
       
               [[-0.1981,  1.5641, -1.2432],
                [-0.9128,  1.3244,  0.2030]]]),
    x: Batch(
           c: tensor([[[-0.0952, -0.4260, -0.0629, -0.9232],
                       [-1.0206,  0.2144,  1.0818, -1.4281],
                       [-1.5229,  1.0045, -0.6346, -0.7571]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1981,  1.5641, -1.2432],
               [-0.9128,  1.3244,  0.2030],
               [-0.1981,  1.5641, -1.2432],
               [-0.9128,  1.3244,  0.2030],
               [-0.1981,  1.5641, -1.2432],
               [-0.9128,  1.3244,  0.2030],
               [-0.1981,  1.5641, -1.2432],
               [-0.9128,  1.3244,  0.2030],
               [-0.1981,  1.5641, -1.2432],
               [-0.9128,  1.3244,  0.2030],
               [-0.1981,  1.5641, -1.2432],
               [-0.9128,  1.3244,  0.2030],
               [-0.1981,  1.5641, -1.2432],
               [-0.9128,  1.3244,  0.2030],
               [-0.1981,  1.5641, -1.2432],
               [-0.9128,  1.3244,  0.2030]]),
    x: Batch(
           c: tensor([[-0.0952, -0.4260, -0.0629, -0.9232],
                      [-1.0206,  0.2144,  1.0818, -1.4281],
                      [-1.5229,  1.0045, -0.6346, -0.7571],
                      [-0.0952, -0.4260, -0.0629, -0.9232],
                      [-1.0206,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 330 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 308 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
